In [1]:
!pip install pyminizip

In [2]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' executed in {end_time - start_time} seconds.")
        return result
    return wrapper

In [3]:
import os
import pyminizip

def create_password_protected_zip_from_folder(output_zip_file, password, folder_path, compression_level=5):
    """
    Creates a password-protected ZIP file containing all files from a specified folder.

    Args:
    output_zip_file: The name of the output ZIP file.
    password: Password for the ZIP file.
    folder_path: Path of the folder containing files to be added to the ZIP file.
    compression_level: ZIP compression level (1-9).
    """
    input_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            input_files.append(file_path)

    if input_files:
        pyminizip.compress_multiple(input_files, [], output_zip_file, password, compression_level)
        print(f"ZIP file '{output_zip_file}' created successfully.")
    else:
        print("No files found in the specified folder.")

In [4]:
import itertools
import string
import zipfile
import concurrent
from concurrent.futures import ThreadPoolExecutor, as_completed

def attempt_password(zip_file_path, password, extract_to_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(path=extract_to_folder, pwd=bytes(password, 'utf-8'))
            return True
    except RuntimeError:
        # This exception is raised for incorrect password
        return False
    except zipfile.BadZipFile:
        # This exception is raised for other zip file related errors
        #print(f"Zip file error with password: {password}")
        return False
    except Exception as e:
        # General exception to catch any other errors
        #print(f"Unexpected error with password '{password}': {e}")
        return False

"""
def attempt_password(zip_file_path, password, extract_to_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(path=extract_to_folder, pwd=bytes(password, 'utf-8'))
            print(f"Success! The password is: {password}")
            return True
    except  (RuntimeError, zipfile.BadZipFile):
        return False
"""
def generate_passwords(chars, length):
    return (''.join(guess) for guess in itertools.product(chars, repeat=length))

@timeit
def brute_force_zip(zip_file_path, extract_to_folder,password_len,  max_workers=4):
    #chars = string.ascii_uppercase + string.ascii_lowercase + '@#'
    chars =  string.ascii_lowercase 
    password_length = password_len  # Adjust as needed

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        password_generator = generate_passwords(chars, password_length)
        future_to_password = {executor.submit(attempt_password, zip_file_path, password, extract_to_folder): password for password in password_generator}
        for future in concurrent.futures.as_completed(future_to_password):
            password = future_to_password[future]
            if future.result():
                print(f"Found password: {password}")
                return




In [5]:
output_zip_file = '/tf/Ethical/example1.zip'  # The name of the output ZIP file
password = 'demo1'  # Password for the ZIP file
password_len = len(password)
folder_path = '/tf/Ethical1/'  # Path of the folder to zip

create_password_protected_zip_from_folder(output_zip_file, password, folder_path)

ZIP file '/tf/Ethical/example.zip' created successfully.


In [6]:
# Example usage
zip_file_path = '/tf/Ethical/example1.zip'
extract_to_folder = '/tf/Ethical/Extract1'

brute_force_zip(zip_file_path, extract_to_folder, password_len)

Found password: demo
Function 'brute_force_zip' executed in 284.06892561912537 seconds.
